# Dashboards and Data Expressions

![QR to GitHub repo](../images/ilgisa-2022-qr.png)

https://github.com/jdcarls2/ilgisa-2022

**Arcade** is Esri's in-house expression language for interacting with all kind of elements. While not exactly JavaScript, folks familiar with JS will feel pretty comfortable with Arcade's structure and logic.

It's been around for a while now, and continues to grow in its capabilities for working with both desktop and web content.

In April of 2021 (and later that same year for Portal users), **ArcGIS Dashboards** added a *huge* new capability: using Arcade to cobble together and manipulate layers and data with a **Data Expression**. The possibilities this has opened up for Dashboards users are hard to overstate.

# Outline

Over the course of this presentation, we will:

- Talk about what a Data Expression (and the underlying FeatureSet) is
- Show the essential parts of a Data Expression
- Build a wide array of examples

# What is a Data Expression?

The Esri blog has a [nice overview](https://www.esri.com/arcgis-blog/products/ops-dashboard/announcements/introducing-data-expressions-in-arcgis-dashboards/) on the subject, but here's the short version:

A Data Expression is *any* Arcade expression that returns a valid FeatureSet, to be used as a new layer in a Dashboard.

## So, What is a FeatureSet?

Again, the [Esri docs](https://developers.arcgis.com/arcgis-rest-js/api-reference/arcgis-rest-feature-service/IFeatureSet) are invaluable here, and I will not be reiterating them in their entirety.

Of the possible properties, three are essential:

```js
{
    fields: [],
    features: [],
    geometryType: 'GeometryType'
}
```

[GeometryTypes](https://developers.arcgis.com/arcgis-rest-js/api-reference/arcgis-rest-feature-service/GeometryType)

### Fields

This is how a FeatureSet's schema is defined. Each field can itself have a number of attributes.

Again, only a few are essential:

```js
[
    {
        name: 'fieldName',
        type: 'FieldType'
    },
    {
        name: 'another_field',
        type: 'FieldType'
    },
    ...
]
```

[FieldTypes](https://developers.arcgis.com/arcgis-rest-js/api-reference/arcgis-rest-request/FieldType)

### Features

Your individual Features need to have a dict of `attributes`. If your FeatureSet is spatial, you'll need to include `geometry` as well.

```js
[
    {
        attributes: {
            fieldName: 'value',
            another_field: 'some other value'
        }
    },
    {
        attributes: {
            fieldName: 'a different value',
            another_field: null
        }
    },
    ...
]
```

# How Do I build a Data Expression?

A Data Expression can be nearly *anything*, provided that it returns a **FeatureSet**. Certain Arcade functions, like `GroupBy` and `Distinct` will return a FeatureSet directly, and make for quick and easy expressions.

More commonly, a Data Expression will need you to build the thing yourself, following the essential properties mentioned earlier.

Start by creating a **dictionary** and establishing the schema in the `fields` property.

```js
var out_dict = {
    fields: [
        {name: 'field_A', type: 'esriFieldTypeInteger'},
        {name: 'field_B', type: 'esriFieldTypeString'}
    ],
    features: [],
    geometryType: ''
}
```

### Populating the `features` Array

Once you create your dict, you'll need to add features to it. In Arcade (as in JS), there is an array function `Push`. Use this to add new elements to an existing array. (Remember, the `features` property of our dict is an empty array!)

This is pretty much always done one feature at a time by means of a loop. Like:

```js
for (var f in input_fs){
    Push(
        out_dict['features'],
        { attributes: {
            field_A: 8,
            field_B: 'some string'
        }}
    )
}
```

## Bringing in Data

To actually *do* anything in our expression, we need some data! The only way to get data into your expression is to use `FeatureSetByPortalItem`.

This function works like this:

```js
FeatureSetByPortalItem(
    Portal('your portal url'),
    'itemID of your service',
    0,                        // or whatever layer index you need
    ['list', 'of', 'fields'], // or ['*'] for all
    false
)
```

### A note on geometry and fields

The way Data Expressions work can be kind of interesting.

If I bring in my layer through `FeatureSetByPortalItem`, then proceed to filter the FeatureSet with the `Filter` function, or perform a spatial overlay operation against it, my browser is actually submitting a request back to the *original service* to get the filtered / intersected set, as opposed to whittling down some in-memory array of features. In other words, the work is done by the server, not the client.

However, if my FeatureSet has been modified, the query is made against an in-memory array of features.

**Not all functions exhibit this behavior!**

### Why does that matter?

When your expression sends a query to the source layer, you have **full access to attributes and geometry**, whether or not those things are actually included in your FeatureSet! This can allow you to create new calculated fields and perform overlay operations without the processing cost of actually bringing that data into your expression.

# Let's See Some Examples Already!

This stuff can be hard to follow just as text. Let's just look at some example expressions to better understand how they're built and how they can be used.

We'll be looking at [a sample dashboard](https://kendall.maps.arcgis.com/apps/dashboards/1b0c8a757ad04bdea4ac502041a673b7) I built for this presentation. It's fully public and uses **Living Atlas** layers, so anyone interested can view and copy the dashboard.

![Dashboard QR](../images/data-expressions-qr.png)